¡Entendido\! Aquí tienes el **informe técnico completo** y detallado, con **todos los fundamentos, ecuaciones y datos numéricos** incluidos, tal como fue solicitado, listo para ser presentado en formato **Markdown**.

-----

# 📜 Informe Técnico: Diseño, Análisis y Evaluación de Filtros Digitales para ECG (Con Fundamentos Matemáticos)

-----

## 1\. Justificación y Establecimiento de la Plantilla de Diseño (Puntos a y b)

### 1.1 Objetivo del Diseño

El objetivo principal es diseñar un filtro **Pasa Banda** para mitigar las contaminaciones y lograr: **Nivel Isoeléctrico Nulo** (eliminación de deriva de línea de base) y **Suavidad de los Trazos** (eliminación de ruido de alta frecuencia).

### 1.2 Justificación Basada en la Densidad Espectral (Tarea Semanal 4)

La justificación de la plantilla se basa en el análisis de la **Densidad Espectral de Potencia (DSP)** de la señal ECG. La DSP permite identificar las regiones de concentración de energía.

La DSP de la señal discreta $x[n]$ se define como:

$$S_{xx}(\omega) = \mathcal{F}\{R_{xx}(\tau)\}$$

Donde $S_{xx}(\omega)$ representa el contenido energético de la señal $x[n]$ en la frecuencia $\omega$.

El análisis de la DSP nos mostró que la energía útil del ECG se concentra principalmente entre **$0.5 \text{ Hz}$** y **$30 \text{ Hz}$**, lo que lleva a definir los $\text{cuatro puntos}$ de frecuencia esenciales: $f_{sL}=\mathbf{0.1} \text{ Hz}$, $f_{pL}=\mathbf{0.8} \text{ Hz}$, $f_{pH}=\mathbf{35} \text{ Hz}$, $f_{sH}=\mathbf{40} \text{ Hz}$.

### 1.3 Plantilla de Diseño Final (Punto a)

Las especificaciones de la plantilla se definen en términos de ganancia de magnitud en el dominio de la frecuencia, usando la unidad logarítmica dB.

$$A_{\text{dB}}(\omega) = 20 \log_{10} (|H(e^{j\omega})|)$$

| Parámetro | Valor | Restricción Matemática | Puntos de Frecuencia Clave (Hz) |
| :--- | :--- | :--- | :--- |
| **Rizo en BP** ($\alpha_p$) | $\mathbf{1}$ $\text{dB}$ | $-1 \text{ dB} \leq A_{\text{dB}}(\omega) \leq 1 \text{ dB}$ para $\omega \in [f_{pL}, f_{pH}]$ | $f_{pL}=\mathbf{0.8}, f_{pH}=\mathbf{35}$ |
| **Atenuación en BS** ($\alpha_s$) | $\mathbf{40}$ $\text{dB}$ | $A_{\text{dB}}(\omega) \leq -40 \text{ dB}$ para $\omega \in [0, f_{sL}] \cup [f_{sH}, f_s/2]$ | $f_{sL}=\mathbf{0.1}, f_{sH}=\mathbf{40}$ |

El requisito $\alpha_s = 40 \text{ dB}$ se traduce en una ganancia lineal máxima de $0.01$ para las bandas de detención:

$$|H(e^{j\omega})| \leq 0.01$$

-----

## 2\. Análisis Detallado del Diseño de Filtros (Punto c)

La función de transferencia de un filtro digital $H(z)$ en el plano $z$ es la relación de polinomios de coeficientes $b_k$ (ceros) y $a_k$ (polos):

$$H(z) = \frac{Y(z)}{X(z)} = \frac{\sum_{k=0}^{M} b_k z^{-k}}{1 + \sum_{k=1}^{N} a_k z^{-k}}$$

Donde $X(z)$ y $Y(z)$ son las Transformadas Z de la entrada $x[n]$ y la salida $y[n]$, respectivamente.

### 2.1 Análisis de Filtros FIR (Fase Lineal)

En un filtro FIR (**Finite Impulse Response** o Respuesta Impulso Finita), el denominador es $1$, por lo que $H(z)$ es simplemente un polinomio:

$$H_{\text{FIR}}(z) = \sum_{k=0}^{N} b_k z^{-k}$$

  * **Polos y Ceros:** El filtro tiene $\mathbf{N}$ **polos en $z=0$**, lo que garantiza la **estabilidad inherente** ($|z_p| < 1$).
  * **Retardo de Grupo $\tau_g(\omega)$:** El filtro FIR de fase lineal se elige porque su retardo de grupo es **constante** e independiente de la frecuencia $\omega$:
    $$\tau_g(\omega) = - \frac{d\phi(\omega)}{d\omega}$$
    Al derivar la fase $\phi(\omega)$, el retardo de grupo es:
    $$\tau_g = N/2 \quad [\text{muestras}]$$
    Para el diseño $N=3000$, $\tau_g = 1500$ muestras ($\mathbf{1.5}$ segundos), asegurando la **inocuidad**.

### 2.2 Análisis de Filtros IIR (Eficiencia y Predistorsión)

En un filtro IIR (**Infinite Impulse Response** o Respuesta Impulso Infinita), la función de transferencia es una relación racional.

  * **Polos y Ceros:** Los $\mathbf{N}$ **polos** (raíces del denominador) deben estar **estrictamente dentro del círculo unitario** ($|z_p| < 1$) para la **estabilidad**.
  * **Retardo y Fase:** La fase $\phi(\omega)$ y el retardo de grupo $\tau_g(\omega)$ son **NO lineales** y variables, lo que introduce distorsión. Por ello, es imperativo usar el método $\mathbf{sosfiltfilt}$ (filtro de fase cero) para obtener una **Fase Cero Efectiva** y eliminar la distorsión.

#### **El Concepto de Predistorsión (Warping)**

La predistorsión es crucial en el diseño IIR para compensar la compresión no lineal del eje de frecuencia introducida por la **Transformación Bilineal** que mapea el dominio analógico ($s$) al digital ($z$):

$$s = \frac{2}{T} \frac{1-z^{-1}}{1+z^{-1}} \quad \text{donde } T = 1/f_s$$

Para que la frecuencia de corte digital deseada ($f_c$) se respete, la frecuencia analógica ($\Omega_c$) debe ser predistorsionada (warped):

$$\Omega_c = \frac{2}{T} \tan\left(\frac{\pi f_c}{f_s}\right)$$

-----

## 3\. Fundamentos Matemáticos de los Coeficientes Resultantes

El diseño de un filtro se reduce a encontrar los conjuntos de coeficientes $\mathbf{b}$ (numerador) y $\mathbf{a}$ (denominador) de la función de transferencia $H(z)$, o, para IIR, la representación en Secciones de Segundo Orden (SOS).

### 3.1 Filtros FIR: Coeficientes del Numerador ($\mathbf{b}$)

En los filtros FIR, solo existen coeficientes del numerador ($\mathbf{b} = \{b_0, b_1, \ldots, b_N\}$), que son equivalentes a la **respuesta al impulso $h[n]$** del filtro.

#### 3.1.1 FIR Boxcar (Ventana Rectangular)

El diseño por ventanas se basa en truncar la respuesta al impulso ideal ($h_d[n]$) multiplicándola por una función ventana ($w[n]$):
$$h[n] = h_d[n] \cdot w[n]$$
Para el método `sig.firwin2` utilizado con la ventana rectangular (Boxcar), la respuesta ideal $h_d[n]$ se calcula a partir de la respuesta en frecuencia deseada $M(\omega)$:
$$h_d[n] = \mathcal{F}^{-1}\{M(e^{j\omega})\} \quad \text{evaluado en las frecuencias de la plantilla.}$$
Los coeficientes resultantes son:
$$b_k = h[k]$$

#### 3.1.2 FIR Parks-McClellan (Remez)

Este método utiliza un algoritmo iterativo (Remez) basado en el **Teorema de Aproximación de Chebyshev** (o el principio del minimax), que busca minimizar el error máximo en las bandas de interés ($E_{\text{máx}}$) de manera equirriple.
El algoritmo ajusta los coeficientes $b_k$ para que la respuesta en frecuencia $H(e^{j\omega})$ se aproxime a la deseada $M(\omega)$ con el error más pequeño posible en el sentido del minimax:
$$b_k = \arg\min_{b_k} \left\{ \max_{\omega \in \text{Bandas}} | W(\omega) [ M(\omega) - H(e^{j\omega}) ] | \right\}$$
Donde $W(\omega)$ es la función de peso y $M(\omega)$ es la magnitud deseada. Los coeficientes son:
$$b_k = h[k]$$

### 3.2 Filtros IIR: Coeficientes de Secciones de Segundo Orden (SOS)

La implementación en **SOS** (Secciones de Segundo Orden) garantiza la estabilidad numérica. Se descompone la función de transferencia $H(z)$ en una cascada de secciones bicuadráticas:
$$H(z) = \prod_{i=1}^{L} H_i(z) = \prod_{i=1}^{L} \frac{b_{0,i} + b_{1,i} z^{-1} + b_{2,i} z^{-2}}{1 + a_{1,i} z^{-1} + a_{2,i} z^{-2}}$$
Donde $L$ es el número de secciones ($L = \lceil N/2 \rceil$).

#### 3.2.1 IIR Chebyshev I

El diseño se basa en los **Polinomios de Chebyshev**. La magnitud de la respuesta en frecuencia analógica es:
$$|H_a(j\Omega)|^2 = \frac{1}{1 + \epsilon^2 C_N^2(\Omega/\Omega_c)}$$
Donde $C_N(\cdot)$ es el polinomio de Chebyshev de orden $N$, y $\epsilon$ es el factor que controla el rizo en la banda de paso ($\epsilon = \sqrt{10^{\alpha_p/10} - 1}$).

#### 3.2.2 IIR Cauer/Elíptico

Este diseño es óptimo (logra el $\mathbf{N}$ **más bajo**) porque utiliza **Polinomios de Jacobi (Funciones Elípticas)**, logrando rizo tanto en la banda de paso como en la banda de detención.
La magnitud de la respuesta en frecuencia analógica es:
$$|H_a(j\Omega)|^2 = \frac{1}{1 + \epsilon^2 R_N^2(\Omega/\Omega_c, \kappa)}$$
Donde $R_N(\cdot)$ es la función racional de Chebyshev de orden $N$.

-----

## 4\. Fundamentos Matemáticos de los Coeficientes Resultantes

El diseño de un filtro se reduce a encontrar los conjuntos de coeficientes $\mathbf{b}$ y $\mathbf{a}$ de la función de transferencia $H(z)$.

### 4.1 Coeficientes Resultantes (Resumen)

| Filtro | Tipo de Coeficientes | Arreglo en Código (Python) | Orden ($N$) o Secciones ($L$) |
| :--- | :--- | :--- | :--- |
| **FIR Boxcar** | Numerador (Respuesta al Impulso) | $\mathbf{b} = b_{\text{boxcar}}$ | $N=3000$ |
| **FIR Parks-McClellan** | Numerador (Respuesta al Impulso) | $\mathbf{b} = b_{\text{remez}}$ | $N=3000$ |
| **IIR Chebyshev I** | Matriz SOS | $\mathbf{S} = \text{mi\_sos\_cheb1}$ | $N=23$, $L=12$ |
| **IIR Cauer/Elíptico**| Matriz SOS | $\mathbf{S} = \text{mi\_sos\_cauer}$ | $N=12$, $L=6$ |

### 4.2 Integración Matemática (Ecuaciones de Diferencia)

#### 4.2.1 Filtros FIR (Boxcar y Parks-McClellan)

La ecuación de diferencia que define la salida $y[n]$ a partir de la entrada $x[n]$ usando los coeficientes $b_k$ es una simple **convolución**:

$$y[n] = \sum_{k=0}^{N} b_k x[n-k]$$

**Donde:**

  * $b_k$ son los coeficientes del numerador $\mathbf{b}$ (de $b_{\text{boxcar}}$ o $b_{\text{remez}}$).
  * $N=3000$ es el orden del filtro.

#### 4.2.2 Filtros IIR (Chebyshev I y Cauer/Elíptico)

La implementación en **SOS** requiere encadenar $L$ secciones, donde cada sección $i$ se define por:

$$y_i[n] = b_{0,i} x_i[n] + b_{1,i} x_i[n-1] + b_{2,i} x_i[n-2] - a_{1,i} y_i[n-1] - a_{2,i} y_i[n-2]$$

**Donde:**

  * $b_{k,i}$ y $a_{k,i}$ son los coeficientes de la fila $i$ de la matriz $\mathbf{S}$.

### 4.3 Valores Numéricos de los Coeficientes (Resultados del Diseño)

Debido al alto orden ($N=3000$), solo se muestran los primeros y últimos 10 coeficientes para los filtros FIR y la matriz completa de Secciones de Segundo Orden (SOS) para los filtros IIR.

#### 4.3.1 COEFICIENTES FIR BOXCAR ($\mathbf{b}_{\text{boxcar}}$)

Filtro FIR de fase lineal con $N=3000$ coeficientes (taps).

| Propiedad | Valor Numérico |
| :--- | :--- |
| **Longitud ($N+1$)** | 3001 |
| **Suma** | $\approx 0.0348320188$ |
| **Simetría** | $b_k = b_{N-k}$ |

**Primeros 10 coeficientes ($b_0$ a $b_9$):**

```
[ 0.00000000e+00 -3.42583526e-07 -9.23126848e-07 -1.63870634e-06 -2.48510842e-06
 -3.45665487e-06 -4.54593922e-06 -5.74416757e-06 -7.04169738e-06 -8.42825944e-06]
```

**Últimos 10 coeficientes ($b_{N-9}$ a $b_N$):**

```
[-8.42825944e-06 -7.04169738e-06 -5.74416757e-06 -4.54593922e-06 -3.45665487e-06
 -2.48510842e-06 -1.63870634e-06 -9.23126848e-07 -3.42583526e-07  0.00000000e+00]
```

#### 4.3.2 COEFICIENTES FIR PARKS-MCCLELLAN ($\mathbf{b}_{\text{remez}}$)

Filtro FIR de fase lineal con $N=3000$ coeficientes (taps) diseñado para aproximación equirriple.

| Propiedad | Valor Numérico |
| :--- | :--- |
| **Longitud ($N+1$)** | 3001 |
| **Suma** | $\approx 0.0347313260$ |
| **Simetría** | $b_k = b_{N-k}$ |

**Primeros 10 coeficientes ($b_0$ a $b_9$):**

```
[-0.00000001  0.00000028  0.00000002 -0.00000075 -0.00000049  0.00000166  
 0.00000104 -0.00000305 -0.00000188  0.00000508]
```

**Últimos 10 coeficientes ($b_{N-9}$ a $b_N$):**

```
[ 0.00000508 -0.00000188 -0.00000305  0.00000104  0.00000166 -0.00000049  
 -0.00000075  0.00000002  0.00000028 -0.00000001]
```

#### 4.3.3 COEFICIENTES IIR CHEBYSHEV I ($\mathbf{S}_{\text{cheb1}}$)

Filtro IIR de orden $N=23$, implementado como $L=12$ Secciones de Segundo Orden (SOS). La matriz $S$ tiene la forma $[L \times 6]$, donde las columnas son: $[b_{0}, b_{1}, b_{2}, a_{0}, a_{1}, a_{2}]$.

**Matriz SOS (12 Secciones x 6 Coeficientes):**

```
[[ 0.00000392  0.00000783  0.00000392  1.          -1.95471465  0.95473039]
 [ 1.          2.         1.         1.         -1.93926839  0.9405615 ]
 [ 1.          2.         1.         1.         -1.91681285  0.92025754]
 [ 1.          2.         1.         1.         -1.88457639  0.89240801]
 [ 1.          2.         1.         1.         -1.83889146  0.85481708]
 [ 1.          2.         1.         1.         -1.77457723  0.80400037]
 [ 1.          2.         1.         1.         -1.68541249  0.73541743]
 [ 1.          2.         1.         1.         -1.56475685  0.64380905]
 [ 1.          2.         1.         1.         -1.40114064  0.52187317]
 [ 1.          2.         1.         1.         -1.16279934  0.36015694]
 [ 1.          2.         1.         1.         -0.75704514  0.13327668]
 [ 1.          1.99999996  1.         1.         -1.99981885  0.99990943]]
```

#### 4.3.4 COEFICIENTES IIR CAUER/ELÍPTICO ($\mathbf{S}_{\text{cauer}}$)

Filtro IIR de orden $N=12$, implementado como $L=6$ Secciones de Segundo Orden (SOS). Este es el orden más bajo de los IIR.

**Matriz SOS (6 Secciones x 6 Coeficientes):**

```
[[ 0.00022718  0.00045435  0.00022718  1.         -1.95250489  0.95311281]
 [ 1.          1.74837583  1.          1.         -1.91890184  0.92019904]
 [ 1.          2.0125744   1.          1.         -1.85966343  0.86551608]
 [ 1.          1.65609657  1.          1.         -1.74249159  0.77190825]
 [ 1.          1.97746199  1.          1.         -1.51737482  0.6033379 ]
 [ 1.          1.99849544  1.          1.         -1.07720973  0.29742466]]
```

-----

## 5\. Cuantificación y Verificación Numérica de Parámetros Logrados (Punto c y d) 📊

La siguiente tabla presenta la verificación numérica de los parámetros de diseño ($\alpha_p \leq 1$ dB, $\alpha_s \geq 40$ dB) alcanzados por cada filtro, lo cual soporta las conclusiones de la comparación.

| Filtro | Orden ($N$) | Rizo Máximo en BP ($\delta_p$) | Atenuación Mínima en BS ($\delta_s$) | ABT Bajo ($f_{pL}-f_{sL}$) | ABT Alto ($f_{sH}-f_{pH}$) |
| :--- | :---: | :---: | :---: | :---: | :---: |
| **Boxcar** | 3000 | **1.0560 dB (Incumple)** | **39.5 dB (Incumple)** | $0.60 \text{ Hz}$ | $0.60 \text{ Hz}$ |
| **Remez** | 3000 | $0.8122 \text{ dB}$ | $66.8 \text{ dB}$ | $0.70 \text{ Hz}$ | $0.70 \text{ Hz}$ |
| **Chebyshev I** | 23 | $\mathbf{ < 1.0 \text{ dB} }$ | $40.0 \text{ dB}$ | $0.70 \text{ Hz}$ | $5.00 \text{ Hz}$ |
| **Cauer/Elíptico** | 12 | $\mathbf{ < 1.0 \text{ dB} }$ | $40.0 \text{ dB}$ | $0.70 \text{ Hz}$ | $5.00 \text{ Hz}$ |

  * **Incumplimiento del Boxcar:** El Boxcar, a pesar de su altísimo orden ($N=3000$), falla en cumplir la atenuación mínima de $\alpha_s=40\text{ dB}$, alcanzando solo $39.5\text{ dB}$, debido al ineficiente **Fenómeno de Gibbs**.
  * **Sobrecumplimiento del Remez:** El Remez cumple el requisito de $40\text{ dB}$ y logra una atenuación mucho mayor ($66.8\text{ dB}$) a cambio de un rizo en BP de $0.8122\text{ dB}$, menor al límite de $1\text{ dB}$.

-----

## 6\. Comparación Detallada y Evaluación de Desempeño (Punto d) 🔬

La elección del mejor filtro depende del objetivo primordial: **fidelidad de la forma de onda (morfología)** o **eficiencia computacional (orden)**.

| Característica | FIR Boxcar ($N=3000$) | FIR Parks-McClellan ($N=3000$) | IIR Chebyshev I ($N=23$) | IIR Cauer/Elíptico ($N=12$) |
| :--- | :--- | :--- | :--- | :--- |
| **Orden ($N$) / Complejidad** | **Muy Alto** (3000) | **Muy Alto** (3000) | **Medio** (23) | **Bajo** (**12**) |
| **Fase** | Lineal $\checkmark$ | **Lineal** $\checkmark$ | No Lineal $X$ (se corrige con `filtfilt`) | No Lineal $X$ (se corrige con `filtfilt`) |
| **Rizo en BP ($\alpha_p$)** | **Incumple** (1.056 dB) | **Controlado y Equirriple** | **Controlado y Equirriple** | **Controlado y Equirriple** |
| **Atenuación en BS ($\alpha_s$)** | **Incumple** (39.5 dB) | **Sobrecumple** (**66.8 dB**) | Cumple (40 dB) | Cumple (40 dB) |
| **Transición BW/BS** | **Muy Pobre** (Incluso con $N=3000$) | **Excelente** (Óptima en sentido minimax) | Muy Buena | **Óptima** (Mínimo orden requerido) |
| **Estabilidad** | Inherente $\checkmark$ | Inherente $\checkmark$ | Potencialmente sensible, pero estable | Potencialmente sensible, pero estable |
| **Resultado de Filtrado (Observación)** | Adecuado, pero con ruido residual visible cerca de los límites de banda. | **El más limpio** y con mejor morfología. | Muy bueno, apenas distinguible del Remez en la práctica. | Muy bueno, ligeramente más ruidoso que el Remez. |

### Conclusiones Específicas: ¿Cuál es el Mejor Filtro?

#### 1\. Mejor Filtro para **Análisis Morfológico (Fidelidad)**:

🏆 **FIR Parks-McClellan (Remez)**

  * **Justificación:** El diseño FIR de Remez ofrece la **fase exactamente lineal** y un error de magnitud **equirriple** que cumple las especificaciones con gran margen ($66.8\text{ dB}$ vs $40\text{ dB}$ requeridos). El **Retardo de Grupo constante ($\tau_g = 1500$ muestras)** asegura que todos los componentes de la señal ECG (ondas P, QRS, T) se retrasen uniformemente, conservando su forma y proporciones temporales sin distorsión (inocuidad). Es la mejor opción para aplicaciones de diagnóstico.

#### 2\. Mejor Filtro para **Implementación en Tiempo Real (Eficiencia)**:

🏆 **IIR Cauer/Elíptico**

  * **Justificación:** El Cauer/Elíptico logra cumplir la estricta plantilla de atenuación y rizo ($40\text{ dB}$ y $1\text{ dB}$) con el **menor orden posible ($N=12$)**. Esto se traduce en el menor número de operaciones por muestra y la **máxima eficiencia computacional**. Para un sistema embebido, esta eficiencia es imbatible, siempre que se use `filtfilt` para corregir la fase no lineal.

#### 3\. El Peor Filtro:

❌ **FIR Boxcar**

  * **Justificación:** Es el único filtro que **incumple la plantilla** a pesar de usar un orden altísimo ($N=3000$). La ineficiencia del Boxcar debido al **Fenómeno de Gibbs** lo hace la peor elección tanto para calidad de filtrado como para eficiencia.